In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./inu_ER.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
document_list

[Document(metadata={'source': './inu_ER.docx'}, page_content='인천대학교 학칙 시행세칙\n\n인천대학교 학칙 시행세칙\n\n[시행 2024.8.6] [인천대학교학교시행세칙 제181호, 2024.8.6, 일부개정]\n\n\n\n\n\n인천대학교 인천대학교 교무과 032-835-9218\n\n\n\n제1장 총칙\n\n\n\n제1조(목적) 이 세칙은 「인천대학교(이하 “본교”라 한다) 학칙」(이하 “학칙”이라 한다) 시행에 따른 필요한 세부사항을 규정함을 목적으로 한다.\n\n\n\n제2장 입학\n\n\n\n제2조(입학의 정의) 입학이란 본교 각 대학에 신입학·재입학·편입학하는 것을 총칭하며, 본교 학생으로서 수강할 수 있는 자격을 얻는 것을 말한다.\n\n\n\n제3조(신·편입생 모집요강) 총장은 매 학년도 신·편입생의 선발기준과 전형방법 및 모집인원 등에 관한 사항을 정하여 공고한다. <개정 2024.8.6.>\n\n\n\n제4조(신·편입학 허가 절차) <개정 2024.8.6.>\n\n<항삭제 2024.8.6.>  ① 신·편입생 선발은 해당 전형별 성적순으로 합격자를 사정하고 총장이 위촉하는 위원으로 구성하는 대학입학전형관리위원회의 심의를 거쳐 총장이 최종적으로 결정한다.<개정 2024.8.6.>\n\n② 합격자 발표 시 예비 합격후보자 약간 명을 사정하여 공고할 수 있다.<개정 2024.8.6.>\n\n③ 신·편입학 허가 절차에 관한 세부사항은 당해 학년도 신·편입학전형 시행계획을 통해 따로 정한다.<개정 2024.8.6.>\n\n\n\n제4조의2(입학취소) ① 다음 각 호의 어느 하나에 해당되는 경우에는 입학 및 졸업 후에라도 입학허가를 취소할 수 있다. 이 경우 학적을 말소하며, 제출한 서류와 등록금은 반환하지 아니한다. 단, 등록금을 납부한 학기가 종료되지 않은 경우에는 「대학 등록금에 관한 규칙」 별표 2의 반환기준에 준하여 반환한다. \n\n1. 지원자격을 위반한 경우\n\n2. 지원자 귀책사유로 입학원

In [9]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = UpstageEmbeddings(model='solar-embedding-1-large')

In [10]:
# from dotenv import load_dotenv
# from langchain_openai import OpenAIEmbeddings

# # 환경변수를 불러옴
# load_dotenv()

# # OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
# embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [11]:
embedding

UpstageEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7f6a7de30df0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7f6a7de33d00>, model='solar-embedding-1-large', dimensions=None, upstage_api_key=SecretStr('**********'), upstage_api_base='https://api.upstage.ai/v1/solar', embedding_ctx_length=4096, embed_batch_size=10, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, http_client=None, http_async_client=None)

In [13]:
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때 
database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-inu-new', persist_directory="./chroma_inu-new")

# database = Chroma(collection_name='chroma-tax', persist_directory="./chroma_inu", embedding_function=embedding)

In [14]:
query = '졸업학점은 몇 점 인가요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
# retrieved_docs = database.similarity_search(query, k=3)
retriever = database.as_retriever(    search_kwargs={"k": 3},)

In [15]:
retriever.invoke(query)

[Document(id='f10d114c-cde4-4b4a-bde2-3127c33331c9', metadata={'source': './inu_ER.docx'}, page_content='② 학사과정 시 본교 대학원의 교과목을 최대 12학점까지 별도로 취득할 수 있으며 그 학점 및 성적은 대학원에서 관리한다.<개정 2023.9.1.>\n\n③ 제2항에 따라 취득한 학점은 최대 6학점의 범위에서 학사과정 전공심화 학점으로 중복하여 인정할 수 있다. 다만, 학과에서 석사과정 연계 전공과 유사하다고 판단한 경우에 한하고, 상이하다고 판단한 경우 또는 학생이 학·석사 연계과정을 포기한 경우에는 일반선택 학점으로 인정할 수 있다.<개정 2022.7.8., 2023.9.1.>\n\n④ 연계과정에 선발된 학생은 대학원 등록 전까지 대학원 지도교수의 승인을 받은 연구활동중간보고서를 대학원에 제출하여야 한다.<개정 2023.9.1., 2024.7.31.>\n\n\n\n제98조(졸업) ① 학사과정의 졸업요건을 충족하고, 취득학점의 평점평균이 3.3 이상인 학생은 7학기 말에 졸업한다. 다만, 7학기 중에 대학원 입학을 위한 등록을 하여야 한다. <개정 2023.9.1.>\n\n② 제98조제1항을 이행하지 않는 학생은 학사과정의 졸업요건을 충족하였음에도 불구하고 제94조에 따라 졸업할 수 없으며, 학부 8학기 등록금을 납부하여야 한다. 다만, 학사과정의 졸업사정 기준에 의한 조기졸업 요건을 충족하는 경우에는 예외로 한다.<개정 2023.9.1.>\n\n③ 7학기에 학사과정의 졸업요건을 충족하고, 취득학점의 평점평균이 3.3 이상인 학생이 7학기 중에 대학원에 입학하지 못한 경우, 학부 8학기 등록금을 납부하고 8학기 중에 대학원 입학을 위한 등록을 하여야 하고, 8학기 말에 졸업하여 학·석사연계과정을 이어갈 수 있다.<신설 2024.7.31.>\n\n\n\n제99조(중도포기) ① 본 과정을 이수 중인 학생이 중도에 포기를 원하는 경우에는 마지막 학기말까지 포기신청서를 대학원 지도

In [26]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [29]:
from langchain import hub

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [30]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [31]:
ai_message = retrieval_chain.invoke({"input": query})

In [35]:
ai_message['answer']

'제공된 정보에서는 졸업학점이 130점부터 140점까지 다양하게 설정되어 있습니다. 정확한 졸업학점은 구체적인 학과나 프로그램에 따라 다를 수 있습니다.'